In [35]:
import os
from constants import *
import subprocess
import json
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account
from getpass import getpass

In [36]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [37]:
load_dotenv()

True

In [38]:
# Generate mnemonic

mnemonic = os.getenv('MNEMONIC')

In [39]:
command = f'./derive -g --mnemonic ="{mnemonic}" --coin="{ETH}" --numderive=5 --cols=index,path,address,privkey,pubkeyhash,xprv,xpub --format=json'

In [40]:
# Run command above to generate a list of child accounts associated with the given mnemonic 
# and coin.
p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
p_status = p.wait()

keys = json.loads(output)
print(keys)

[{'index': 0, 'path': "m/44'/60'/0'/0/0", 'address': '0xCDc849A83772e7752FB41A4F1a86c2036aE78339', 'privkey': '0x9bd2bfcbff4e0709b8a47633986916bee435443955d5f122967046acbeba4deb', 'pubkeyhash': '6ed9bef40a51c15e3a338c7e0d698d736a6c995b', 'xprv': 'xprvA2VVLA6kAYhXawvvRdbhj8yMMEcHsPK2F99wMuFKoDsQjWJQt9aVqKrZoxqMvrQe7ctyn5G6HTZDWpyQB3Y2VYreKTJLXYmZiNW9Z2XcyMW', 'xpub': 'xpub6FUqjfddzvFpoS1PXf8i6Gv5uGSnGr2scN5YAHewMZQPcJdZRgtkP8B3fF7n1vnashdT9dJs8TynQmcvQGKgqecPaURe68n8mZsguVAuzDk'}, {'index': 1, 'path': "m/44'/60'/0'/0/1", 'address': '0xeA6a979A5C4bF0256c5292F386C41867C514cF0d', 'privkey': '0x2234ee537da753ce21d3a40cc566b254975cc2755fcb0c075a6bff715b06dd03', 'pubkeyhash': '428b10d4e521da29352e050a6c85163e1f99ef69', 'xprv': 'xprvA2VVLA6kAYhXcTpKca8jhmXaaqbd2XcU6sLnLdrGsSKpFhaVdSiZbKJmn5BNVKASwHHREgcD35e5qoMV2FR4sJksckwrE2R6EaDWzaSu3bi', 'xpub': 'xpub6FUqjfddzvFppwtnibfk4uUK8sS7RzLKU6GP92FtRmro8VueAz2p97dFdNg7k5QP8cCVTCeQmZiwr3kbkha4UxJVkTMj2D3pteY1DwWX5q5'}, {'index': 2, 'path': "m/44'/60'

In [41]:
# Given a mnemonic, specified coin, and number, derive_wallets() returns the number of accounts 
# specified.
def derive_wallets(mnemonic, coin, number):
    
    command = f"./derive -g --mnemonic ='{mnemonic}' --coin='{coin}' --numderive='{number}' --cols=index,path,address,privkey,pubkeyhash,xprv,xpub --format=json"
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [42]:
# Create class for coin type. Ended up using variables from constants.py.

# class coin:
#    BTC = 'btc'
#    ETH = 'eth'
#    BTCTEST = 'btc-test'

In [43]:
# takes in a coin and private key and generates bytes object.
def priv_key_to_account(coin, privkey):
    if coin == ETH:
        return Account.privateKeyToAccount(privkey)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(privkey)

In [44]:
# Create variables for ETH private key and address for two accounts for testing.
eth_derive = derive_wallets(mnemonic, ETH, 5)

eth_acc = priv_key_to_account(ETH, eth_derive[0]['privkey'])

eth_to_acc = eth_derive[1]['address']

In [45]:
# Create variables for BTC private key and address for two accounts for testing.
btc_derive = derive_wallets(mnemonic, BTCTEST, 2)

btc_acc = priv_key_to_account(BTCTEST, btc_derive[0]['privkey'])

btc_to_acc = btc_derive[1]['address']

In [46]:
# Create ledger of transaction to be sent later.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [47]:
create_tx(ETH, eth_acc, eth_to_acc, 1)

{'from': '0xCDc849A83772e7752FB41A4F1a86c2036aE78339',
 'to': '0xeA6a979A5C4bF0256c5292F386C41867C514cF0d',
 'value': 1,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 2}

In [48]:
btc_tx = create_tx(BTCTEST, btc_acc, btc_to_acc, 0.000001)
btc_tx

'{"unspents":[{"amount":2409728,"confirmations":1,"script":"76a914a4156458e3032ad331313edb1e3957431b49cdfc88ac","txid":"114128bbaa9a670c521e91c88fe0473167d491189eac26b115a389e9dddaf814","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mnkZ3rcUxb68sCovcwKTJWawmdS9RhbvJx",100],["mvUYjrYuRyvczDreo2VoCcfNWzcYRsbCdw",2384316]]}'

In [49]:
# Takes in same parameters as create_tx(), then calls on create_tx to generate a ledger to be 
# signed, and then sent to the recipient, meeting the given coin's requirements.
def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [50]:
# Takes in the sender and recpient's index number as parameters. Potentially useful for 
# transferring funds between child accounts.
def transaction(coin, sender, to, amount):
    
    send_key = derive_wallets(mnemonic, coin, 6)[sender]['privkey']
    send_hash = priv_key_to_account(coin, send_key)
    to_address = derive_wallets(mnemonic, coin, 6)[to]['address']
    
    return send_tx(coin, send_hash, to_address, amount)

In [51]:
transaction(BTCTEST, 0, 1, 0.0001)

In [54]:
btc_key = wif_to_key('cVxFxckGh3kzTdVWLLMsUJMXx2B8hh2Ku67HC13JTYmp341hstCe')
btc_key.get_balance(BTC)

'0.02374416'

In [55]:
send_tx(ETH, eth_acc, eth_to_acc, 1)

0xcb880362594dcef4fa0b16e3decca6e81adf9dbdba5482e0c4a1c5651151eda4


'0xcb880362594dcef4fa0b16e3decca6e81adf9dbdba5482e0c4a1c5651151eda4'